In [1]:
%load_ext rpy2.ipython
%matplotlib inline

First, let's look at Andreas' new class (below, see CGAT/R.py). We can see that the new class, R_with_History, inherits from the rpy2.robjects R class. The major differences is that we create an empty history with each new instance of the class which we update each time we run R code. Then there are a couple of shorthand functions to save the R image and history. 

In [2]:
from rpy2.robjects import R as RBase


class R_with_History(RBase):
    ''' Redefine the RBase class to allow history to be saved'''
    _instance = None

    def __new__(cls):
        c = RBase.__new__(cls)
        cls._instance = c
        c._history = []
        return cls._instance

    def __call__(self, string):
        self._history.append(string)
        RBase.__call__(self, string)

    def saveImage(self, imageFile):
        self["save.image"](file=imageFile)

    def saveHistory(self, historyFile, append=False):
        ''' save history '''

        filetype = "w"
        if append:
            filetype = "a"

        with open(historyFile, filetype) as outf:
            outf.write("\n".join(self._history) + "\n")

We use this class like so:

In [3]:
import CGAT.R as R

In [4]:
RH = R.R_with_History()
RH('''

temp_vector = rnorm(10)
print(temp_vector)
''')
RH.saveImage("./tutorial.Rdata")
RH.saveHistory("./tutorial.Rhist")

 [1] -0.04786724  0.67919814  0.51918615  0.07146713 -2.68074062  0.21189479
 [7]  0.80139319  0.36264181 -1.05463749 -0.09692264


Now we go into R, and load the image and history. Here we use the %R_magic but we could of course use Rstudio etc. 

In [5]:
%%R
load(file="./tutorial.Rdata")
loadhistory("./tutorial.Rhist")

print(history())

print(temp_vector)


R Historytemp_vector = rnorm(10)
print(temp_vector)


NULL
 [1] -0.04786724  0.67919814  0.51918615  0.07146713 -2.68074062  0.21189479
 [7]  0.80139319  0.36264181 -1.05463749 -0.09692264


In order to use this new class, we need to re-write some of the code in e.g Expression.py so that we are running R commands to use objects available in the global namespace. In the example below, we have the previous code to run the first step of DESeq2 analysis where a function was defined with rpy2. The following cell shows how to run this with the new class. First, we need to create some dummy counts and design objects

In [6]:
import CGAT.Counts as Counts
import CGAT.Expression as Expression
import pandas as pd
import numpy as np

counts_df = pd.DataFrame({"A": np.random.randint(1000, size=100), "B": np.random.randint(1000, size=100),
                          "C": np.random.randint(1000, size=100), "D": np.random.randint(1000, size=100)},
                         index=range(0,100))

design_df = pd.DataFrame({"track": ["A", "B", "C", "D"], "group":["wt","wt","KO","KO"],
                          "include":[1,1,1,1], "pair":[0,0,0,0]})

design_df.set_index("track", inplace=True)

# create Counts and ExperimentalDesign objs
counts = Counts.Counts(counts_df)
design = Expression.ExperimentalDesign(design_df)

print counts.table.head()
print design.table.head()

ref_group = "wt"
model = "~group"
contrast = "group"

     A    B    C    D
0  609  558  353  645
1  422  994  993   24
2  180  463  755  789
3  109  360  552  677
4  300  980  441  998
      group  include  pair
track                     
A        wt        1     0
B        wt        1     0
C        KO        1     0
D        KO        1     0


In the cell below we can see how the section of R code used to be run. We define an R function which requires counts and factors input dataframes and returns a "dds" object. There is also some string substituion for the contrast and ref_group variables

In [7]:
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri

from rpy2.robjects import r as R

r_counts = pandas2ri.py2ri(counts.table)
r_factors_df = pandas2ri.py2ri(design.factors)

R('''suppressMessages(library('DESeq2'))''')

buildDds = R('''
function(counts, factors_df){
for(column in colnames(factors_df)){
  factors_df[[column]] = factor(factors_df[[column]])
}
full_model <- formula("%(model)s")
factors_df$%(contrast)s <- relevel(
   factors_df$%(contrast)s, ref="%(ref_group)s")
dds <- suppressMessages(DESeqDataSetFromMatrix(
         countData= counts,
         colData = factors_df,
         design = full_model))
return(dds)
}''' % locals())


r_dds = buildDds(r_counts, r_factors_df)

print(r_dds)

class: DESeqDataSet 
dim: 100 4 
metadata(1): version
assays(1): counts
rownames(100): 0 1 ... 98 99
rowData names(0):
colnames(4): A B C D
colData names(1): group



Now we rewrite this code to use Andreas' class. To do this, we explicitly create the objects in the R namespace. At the end we can save out the history and image if we wish. 

In [8]:
# create r objects
import CGAT.R as R
R = R.R_with_History()

ro.globalenv['counts'] = pandas2ri.py2ri(counts.table)
ro.globalenv['factors_df'] = pandas2ri.py2ri(design.factors)

R('''

for(column in colnames(factors_df)){
    factors_df[[column]] = factor(factors_df[[column]])}

full_model <- formula("%(model)s")

factors_df$%(contrast)s <- relevel(
   factors_df$%(contrast)s, ref="%(ref_group)s")

dds <- suppressMessages(DESeqDataSetFromMatrix(
         countData= counts,
         colData = factors_df,
         design = full_model))
''' % locals())

R.saveHistory("./tutorial_DESeq2.Rhist")
R.saveImage("./tutorial_DESeq2.Rdata")

In [9]:
%%R
load(file="./tutorial.Rdata")
loadhistory("./tutorial_DESeq2.Rhist")

print(history())

print(dds)

R Historyfor(column in colnames(factors_df)){
    factors_df[[column]] = factor(factors_df[[column]])}
full_model <- formula("~group")
factors_df$group <- relevel(
   factors_df$group, ref="wt")
dds <- suppressMessages(DESeqDataSetFromMatrix(
         countData= counts,
         colData = factors_df,
         design = full_model))


NULL
class: DESeqDataSet 
dim: 100 4 
metadata(1): version
assays(1): counts
rownames(100): 0 1 ... 98 99
rowData names(0):
colnames(4): A B C D
colData names(1): group


Of course, in the real analysis we wouldn't save out the image and history until the end of the analysis. Moreover, if we're running this in a script, we would also like to create the R class when we start the script, and save out the history and image when we stop the script. For this reason, the handling of the R class will be done within Experiment.py and default options for history/imaging saving will be added.

In [12]:
counts = Counts.Counts(counts_df)

import CGAT.R as R
R = R.R_with_History()

# plot counts
ro.globalenv['df'] = pandas2ri.py2ri(counts.table)
R('''
print(cor(df))
''')

# subset design
ro.globalenv['df'] = pandas2ri.py2ri(design.table)
R('''
final_design = df[df$include==1,]
''')

R.saveHistory("./tutorial_DESeq2.Rhist")
R.saveImage("./tutorial_DESeq2.Rdata")

            A          B           C          D
A  1.00000000 -0.1553691 -0.08955991 -0.1177167
B -0.15536908  1.0000000  0.11149109 -0.1319077
C -0.08955991  0.1114911  1.00000000 -0.0307937
D -0.11771672 -0.1319077 -0.03079370  1.0000000


In the code above, we run two R commands. In both cases we call the input DataFrame the rather generic, "df". The first command in the history therefore fails using the final objects

In [19]:
%%R
load(file="./tutorial.Rdata")
loadhistory("./tutorial_DESeq2.Rhist")

print(history())

print(cor(df))
final_design = df[df$include==1,]

R Historyprint(cor(df))
final_design = df[df$include==1,]
NULL
Error in cor(df) : 'x' must be numeric


In order to avoid this, we should use more specific, clear variable names and not modify the input objects.